instructions: run entire notebook after specifying directory variable at the top
> model = resnet152

In [39]:
directory = "/home/paperspace/Documents/dc-grocery-outlet-brand/image_splits_v3"

In [40]:
model_path = 'brand_model.pt'

In [41]:
#torchvision.__version__

In [42]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils import data
import torch.backends.cudnn as cudnn
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import random
from model import *
import matplotlib.pyplot as plt
import time
import os
import copy
from PIL import Image

import pprint, pickle


import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm

import warnings
warnings.filterwarnings("ignore")

In [43]:
def list_files(d):
    r = []
    for root, dirs, files in os.walk(d):
        for name in files:
            r.append(os.path.join(root, name))
    return r

In [66]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

# model = models.resnet34(pretrained=True)
model = models.resnet152(pretrained=True) # we do not specify pretrained=True, i.e. do not load default weights
# model.load_state_dict(torch.load('XNAS/trained_models/xnas_small_cifar10.t7'), strict=False)

for param in model.parameters():
    param.requires_grad = False
    
num_ftrs = model.fc.in_features
print(get_out_features())
model.fc = nn.Linear(num_ftrs, 16358)
model = model.to(device)

cuda:0
2369


RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.

In [30]:
# df = pd.read_csv('UPC-Images-Sample-Labeled.csv').drop(['Short Description', 'ItemNumber', 'SubClass', 'Department', 'class_subclass', 'label'], axis = 1)
# df.Brand = df.Brand.str.lower()
# df = df.sort_values(by='Brand', ascending=True)
# no_dupes_df = df.drop_duplicates("ImageList")
# brand_df = no_dupes_df[no_dupes_df.Brand.notna()]
# # brands_10_or_less = brand_df.groupby('Brand').filter(lambda subdf: len(subdf) <= 10)['Brand']

# # more_than_10_df = brand_df[~brand_df['Brand'].isin(brands_10_or_less)]
# brand_df.Brand = brand_df.Brand.str.lower()
# brand_df.Brand = branddf.Brand.str.replace(' ', '')

# brand_to_num = dict(zip(np.unique(more_than_10_df['Brand'].values), np.arange(len(np.unique(more_than_10_df['Brand'].values)))))
# num_to_brand = dict(zip(np.arange(len(np.unique(more_than_10_df['Brand'].values))), np.unique(more_than_10_df['Brand'].values)))
# brand_to_num.update(num_to_brand)

In [31]:
def read_make_image_splits(file):
    """
        given a .pkl file, returns a 3D numpy array
    """

    pkl_file = open(file, 'rb')

    data1 = pickle.load(pkl_file)
    pkl_file.close()

    return data1


In [45]:
def test_model(model, generator):
    pbar = tqdm(val_generator)
    running_corrects = 0
    for step, batch in enumerate(pbar):
        model.eval()
      # inputs = inputs.to(device)
      # labels = labels.to(device)
        inputs, labels = batch
        inputs = inputs.to(torch.float)
        inputs = inputs.to(device)
        labels = labels.to(device)

      # zero the parameter gradients
        optimizer.zero_grad()

      # forward
      # track history if only in train
      # with torch.set_grad_enabled(phase == 'train'):
      
    
      
      #print(tuple(images)[0])
      # print(labels)
        with torch.no_grad():
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            running_corrects += torch.sum(preds == labels.data)
            # print(preds)
      
    epoch_acc = running_corrects / len(val)
    return epoch_acc.item()

## train the model

In [46]:
torchvision.transforms.ToTensor()

ToTensor()

In [47]:
def save_checkpoint(model, optimizer, save_path, epoch):
    torch.save({
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'epoch' : epoch
    }, save_path)

In [48]:
def load_checkpoint(model, optimizer, load_path):
    checkpoint = torch.load(load_path)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    # epoch = checkpoint['epoch']
    return model, optimizer

In [49]:
train_dir = "/home/paperspace/Documents/dc-grocery-outlet-brand/image_splits_v3/train"
val_dir = "/home/paperspace/Documents/dc-grocery-outlet-brand/image_splits_v3/val"
test_dir = "/home/paperspace/Documents/dc-grocery-outlet-brand/image_splits_v3/test"

In [60]:
num_epochs = 1000
best_acc = 0
losses = []
scores = []
best_model_wts = None
model_path = 'brand_model.pt'
start = time.time()
patience = 10
trigger_times = 0
flag = False
train_files = list_files(train_dir)
test_files = list_files(test_dir)
val_files = list_files(val_dir)
og_files = list_files("/home/paperspace/Documents/dc-grocery-outlet-brand/image_splits_v2")
og_files_2 = list_files("/home/paperspace/Documents/dc-grocery-outlet-brand/image_splits_10_or_less")
files = list(zip(train_files, test_files, val_files, og_files, og_files_2))
optimizer = optim.Adam(model.parameters(), lr=0.05)
# model, optimizer = load_checkpoint(model, optimizer, model_path)

In [61]:
# indices = []
# for file in og_files:
#     df = pd.DataFrame(read_make_image_splits(file), columns=["brand", "size", "measure", "class", "long_description", "ImageList", "image_matrix", "image_tensor"])
#     df = df.dropna()
#     indices.append(random.shuffle(list(range(len(df)))))

# less_than_10_indices = []
# for file in og_files_2:
#     df = pd.DataFrame(read_make_image_splits(file), columns=["brand", "size", "measure", "class", "long_description", "ImageList", "image_matrix", "image_tensor"])
#     df = df.dropna()
#     less_than_10_indices.append(random.shuffle(list(range(len(df)))))

In [64]:
def preprocess(file):
    """
        given a file, returns generators for training
    """

    train_df = pd.DataFrame(read_make_image_splits(file[0]), columns=["brand", "size", "measure", "class", "long_description", "ImageList", "image_matrix", "image_tensor"])
    train_df = train_df.dropna()
    train_df_brand = train_df[["image_tensor", "brand"]]
   
    train = list(map(list, zip(train_df_brand["image_tensor"].values, train_df_brand["brand"].values))) 
    
    train_generator = data.DataLoader(train, batch_size=16)
    
    test_df = pd.DataFrame(read_make_image_splits(file[1]), columns=["brand", "size", "measure", "class", "long_description", "ImageList", "image_matrix", "image_tensor"])
    test_df = test_df.dropna()
    test_df_brand = test_df[["image_tensor", "brand"]]
   
    test = list(map(list, zip(test_df_brand["image_tensor"].values, test_df_brand["brand"].values)))
    
    test_generator = data.DataLoader(test, batch_size=16)
    
    val_df = pd.DataFrame(read_make_image_splits(file[2]), columns=["brand", "size", "measure", "class", "long_description", "ImageList", "image_matrix", "image_tensor"])
    val_df = val_df.dropna()
    val_df_brand = val_df[["image_tensor", "brand"]]
   
    val = list(map(list, zip(val_df_brand["image_tensor"].values, val_df_brand["brand"].values)))

    
    val_generator = data.DataLoader(val, batch_size=16)
    
    return train_generator, test_generator, val_generator, val

In [65]:

for epoch in range(num_epochs):
    print('Epoch {}/{}'.format(epoch + 1, num_epochs))
    print('-' * 10)
    
    optimizer = optim.Adam(model.parameters(), lr=0.05)
    # model, optimizer = load_checkpoint(model, optimizer, model_path)
    file_num = 0
    decrease = False
    
    for file in files:
        print('File {}/{}'.format(file_num, len(files)))
        print('-' * 10)
        
        try:
            train_generator, test_generator, val_generator, val = preprocess(file)

            pbar = tqdm(train_generator)
            running_avg_loss = 0
            for step, batch in enumerate(pbar):
                model.train()
                inputs, labels = batch
                inputs = inputs.to(torch.float)
                inputs = inputs.to(device)
                labels = labels.to(device)


                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                # with torch.set_grad_enabled(phase == 'train'):
                '''images = []
                for image_path in inputs:
                    img = Image.open(os.path.join(image_dir, image_path))
                    images.append(transforms.ToTensor()(img))'''

                #print(tuple(images)[0])
                # print(labels)

                outputs = model(inputs)

                # _, preds = torch.max(outputs, 1)
                # print(labels)
                # print(outputs)
                # print(outputs.shape)
                loss = nn.CrossEntropyLoss()(outputs, labels)
                # print(loss)
                # backward + optimize only if in training phase
                loss.backward()
                # print(loss)
                # pbar.set_description(str(loss.item()))
                optimizer.step()
                # avg_loss = (avg_loss + loss.item()) / (step + 1)
                running_avg_loss += loss.item()
                pbar.set_description("Avg Loss: " + str(running_avg_loss / (step + 1)))

                # statistics
                # running_loss += loss.item() * inputs.size(0)
                # running_corrects += torch.sum(preds == labels.data)
                # scheduler.step()

                 #NOTE: need to fix validation because this val_generator only covers 1 file, not all 50
                    
        except(RuntimeError):
            print('number of dims dont match in permute')
            continue
        file_num += 1 
        epoch_acc = test_model(model, val_generator)
        if epoch_acc > best_acc:
            best_acc = epoch_acc
            best_model_wts = copy.deepcopy(model.state_dict())
        if len(losses) >= 50 and decrease == False:
            last_loss = losses[-50]
            if last_loss < running_avg_loss / len(train_generator):
                decrease = True
                trigger_times += 1
                print('Trigger Times:', trigger_times)
                if trigger_times >= patience:
                    flag = True
                    break
            else:
                trigger_times = 0
        print("validation accuracy:", epoch_acc)
        scores.append(epoch_acc)
        losses.append(running_avg_loss / len(train_generator))
        print()
        
    save_checkpoint(model, optimizer, model_path, epoch)
    if flag:
        break


Epoch 1/1000
----------
File 0/50
----------


  0%|                                                   | 0/133 [00:00<?, ?it/s]/pytorch/aten/src/ATen/native/cuda/Loss.cu:247: nll_loss_forward_reduce_cuda_kernel_2d: block: [0,0,0], thread: [0,0,0] Assertion `t >= 0 && t < n_classes` failed.
/pytorch/aten/src/ATen/native/cuda/Loss.cu:247: nll_loss_forward_reduce_cuda_kernel_2d: block: [0,0,0], thread: [1,0,0] Assertion `t >= 0 && t < n_classes` failed.
/pytorch/aten/src/ATen/native/cuda/Loss.cu:247: nll_loss_forward_reduce_cuda_kernel_2d: block: [0,0,0], thread: [2,0,0] Assertion `t >= 0 && t < n_classes` failed.
/pytorch/aten/src/ATen/native/cuda/Loss.cu:247: nll_loss_forward_reduce_cuda_kernel_2d: block: [0,0,0], thread: [3,0,0] Assertion `t >= 0 && t < n_classes` failed.
/pytorch/aten/src/ATen/native/cuda/Loss.cu:247: nll_loss_forward_reduce_cuda_kernel_2d: block: [0,0,0], thread: [6,0,0] Assertion `t >= 0 && t < n_classes` failed.
/pytorch/aten/src/ATen/native/cuda/Loss.cu:247: nll_loss_forward_reduce_cuda_kernel_2d: block: [0,0,

number of dims dont match in permute
File 0/50
----------


  0%|                                                   | 0/133 [00:00<?, ?it/s]


number of dims dont match in permute
File 0/50
----------


...

  0%|                                                   | 0/133 [00:00<?, ?it/s]


number of dims dont match in permute
File 0/50
----------


  0%|                                                   | 0/133 [00:00<?, ?it/s]


number of dims dont match in permute
File 0/50
----------


EOFError: Ran out of input

In [116]:
list(range(50))[]

0

In [ ]:
end = time.time()
print(f"The model took {(end - start) // 60} minutes to train.")

In [ ]:
ex = preprocess("image_splits_v2/split_0.pkl")
tqdm(ex[0])

In [ ]:
# d = dict(enumerate(k.flatten(), 1))
# d = str(d) ## dump as string  (pickle and other packages parse the dump as bytes)

m = ast.literal_eval(example1) ### convert the dict as str to  dict

m = np.fromiter(m.values(), dtype=float) ## convert m to nparray

In [ ]:
a

In [ ]:
np.fromstring(example1)